In [7]:
# !pip install torch
# !pip install torchvision

### 데이터 작업하기

In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

__모든 torchvision의 datasets는 샘플과 정답을 각각 변경하기 위한 `transform`과 `target_transform`의 두 인자를 포함한다.__

In [9]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor())

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor())


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw




Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw




Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw




Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



__dataset을 DataLoader의 인자로 전달한다. 이는 데이터셋을 iterable 객체로 감싸고, 자동화된 배치, 샘플링(sampling), 섞기(shuffle) 및 다중 프로세스로 데이터 불러오기를 지원한다. __

In [10]:
batch_size = 64

# 데이터로더 생성
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print('Shape of X [N, C, H, W]: ', X.shape)
    print('Shape of y: ', y.shape)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64])


### 모델 만들기

In [21]:
# CPU / GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

# 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU())
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print('\n', model)

Using cpu device

 NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


### 모델 매개변수 최적화하기

In [22]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [23]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # 역전파
        optimizer.zero_grad()  # 이전에 얻은 기울기(gradient)를 초기화
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}   [{current:>5d}/{size:>5d}]')

__파이토치는 미분을 통해 얻은 기울기를 이전에 계산된 기울기 값에 누적시키는 특징이 있다. 그렇기 때문에 `optimizer.zero_grad()`를 통해 미분값을 계속 0으로 초기화 시켜줘야 한다.__

> __f-string formatting__
- 문자열 맨 앞에 f를 붙여주고, 중괄호 안에 직접 변수 이름이나 출력하고 싶은 것 넣기
- \>7d의 의미는 d 타입의 출력값에 대하여 7만큼의 너비를 가지며, 오른쪽 정렬이라는 의미

In [24]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [25]:
epochs = 5
for t in range(epochs):
    print(f'Epochs {t+1}\n--------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done!')

Epochs 1
--------------------------
loss: 2.304629   [    0/60000]
loss: 2.297821   [ 6400/60000]
loss: 2.288503   [12800/60000]
loss: 2.288203   [19200/60000]
loss: 2.275687   [25600/60000]
loss: 2.271428   [32000/60000]
loss: 2.257886   [38400/60000]
loss: 2.250818   [44800/60000]
loss: 2.272650   [51200/60000]
loss: 2.227923   [57600/60000]
Test Error: 
 Accuracy: 44.8%, Avg loss: 2.236562 

Epochs 2
--------------------------
loss: 2.254951   [    0/60000]
loss: 2.239935   [ 6400/60000]
loss: 2.210459   [12800/60000]
loss: 2.209432   [19200/60000]
loss: 2.180371   [25600/60000]
loss: 2.190202   [32000/60000]
loss: 2.150356   [38400/60000]
loss: 2.142451   [44800/60000]
loss: 2.207429   [51200/60000]
loss: 2.102525   [57600/60000]
Test Error: 
 Accuracy: 45.1%, Avg loss: 2.123455 

Epochs 3
--------------------------
loss: 2.173542   [    0/60000]
loss: 2.141264   [ 6400/60000]
loss: 2.083161   [12800/60000]
loss: 2.076102   [19200/60000]
loss: 2.013085   [25600/60000]
loss: 2.05922

### 모델 저장하기

In [26]:
torch.save(model.state_dict(), 'model.pth')
print('Save Pytorch Model State to model.pth')

Save Pytorch Model State to model.pth


### 모델 불러오기

In [27]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [29]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
